In [2]:
import torch
import numpy as np

#### 1灵活的初始化方法
支持list或ndarray输入，默认继承原始的数据类型，并且支持指定数据类型。

In [49]:
# 使用 torch.tensor()初始化
a = torch.tensor([[1, -1], [1, -1]])
b = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]))
c = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]], dtype=np.int64))
d = torch.tensor([[1, -1], [1, -1]], dtype=torch.int32)
print(a) 
print('dtype of a is',a.dtype)
print('id of a is',a.data_ptr(),end='\n\n')

print(b)
print('dtype of b is',b.dtype)
print('id of b is',b.data_ptr(),end='\n\n')

print(c)
print('dtype of c is',c.dtype)
print('id of c is',c.data_ptr(),end='\n\n')

print(d)
print('dtype of d is',d.dtype)
print('id of d is',d.data_ptr())

tensor([[ 1, -1],
        [ 1, -1]])
dtype of a is torch.int64
id of a is 3232800068608

tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int32)
dtype of b is torch.int32
id of b is 3232800062208

tensor([[1, 2, 3],
        [4, 5, 6]])
dtype of c is torch.int64
id of c is 3232796620032

tensor([[ 1, -1],
        [ 1, -1]], dtype=torch.int32)
dtype of d is torch.int32
id of d is 3232495737088


In [87]:
# 使用 torch.from_numpy()初始化
a = np.array([1.0, 2, 3])
t = torch.from_numpy(a)
print(t,t.dtype) #会继承dtype,而非使用默认的torch.int64
t[0] = -1
print(a,a.dtype)

tensor([1., 2., 3.], dtype=torch.float64) torch.float64
[-1.  2.  3.] float64


In [7]:
# 使用 torch.as_tensor()初始化
a = np.array([1, 2, 3])
t = torch.as_tensor(a) # 未发生copy
print(t,t.dtype)
t[0] = -1
print(a,a.dtype)

a = np.array([1, 2, 3])
t = torch.as_tensor(a, device=torch.device('cuda')) # 发生了copy
print(t,t.dtype)
t[0] = -1
print(a,a.dtype)

tensor([1, 2, 3], dtype=torch.int32) torch.int32
[-1  2  3] int32
tensor([1, 2, 3], device='cuda:0', dtype=torch.int32) torch.int32
[1 2 3] int32


In [47]:
# 使用 torch.as_tensor()初始化, but for list
a = [9999999999999999999999999999999999999999999999.99999,6.5]
t = torch.as_tensor(a) # 发生copy
# t = torch.as_tensor(a,dtype=torch.float64) # 发生copy
print(t,t.dtype)
t[0] = -1
print(a,type(a))
print(t,t.dtype)

tensor([   inf, 6.5000]) torch.float32
[1e+46, 6.5] <class 'list'>
tensor([-1.0000,  6.5000]) torch.float32


In [45]:
# 使用 torch.as_tensor()初始化, but for ndarray
a = np.array([9999999999999999999999999999999999999999999999.99999,6.5])
t = torch.as_tensor(a) 
print(t,t.dtype)
print(a,a.dtype)


tensor([1.0000e+46, 6.5000e+00], dtype=torch.float64) torch.float64
[1.0e+46 6.5e+00] float64


In [57]:
# torch.as_tensor() but for tensor
b = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]))
print(b)
print('id of b is',b.data_ptr(),end='\n\n')
c = torch.as_tensor(b) 
# c = torch.as_tensor(b,dtype=torch.int64) 
print(c)
print('id of c is',c.data_ptr(),end='\n\n')

tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int32)
id of b is 3232800067264

tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int32)
id of c is 3232800067264



In [89]:
# torch.clone() 可微
a = torch.tensor(np.array([[1.0, 2, 3], [4, 5, 6]]), requires_grad=True)
print(a)
print('id of a is',a.storage().data_ptr(),end='\n\n')

# b = torch.tensor(a)
# b = torch.clone(a)
b = torch.clone(a).detach()
print(a.requires_grad, b.requires_grad)

print(b)
print('id of b is',b.storage().data_ptr(),end='\n\n')

tensor([[1., 2., 3.],
        [4., 5., 6.]], dtype=torch.float64, requires_grad=True)
id of a is 3232796615168

True False
tensor([[1., 2., 3.],
        [4., 5., 6.]], dtype=torch.float64)
id of b is 3232796612608



In [77]:
# 情况一
a = torch.tensor(1.0, requires_grad=True)
b = a.clone()
print(a.data_ptr(), b.data_ptr())
# (2892334894104, 2892334859464)  # 表明a和b不是同一个对象
print(a.requires_grad, b.requires_grad)
# (True, True)  # 两者的requires_grad都是True
c = a * 2
c.backward()
print(a.grad)
# tensor(2.)

d = b * 3
d.backward()
print(b.grad) # b的梯度值为None

print(a.grad)
# tensor(5.)  # b的梯度叠加在a上


3232495737088 3232495737152
True True
tensor(2.)
None
tensor(5.)


d:\software\anaconda\envs\gpt\lib\site-packages\torch\_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten\src\ATen/core/TensorBody.h:417.)
  return self._grad


In [75]:
# 情况二：
a = torch.tensor(1.0, requires_grad=False)
b = a.clone()
print(a.data_ptr(), b.data_ptr())
print(a.requires_grad, b.requires_grad)
# (False False)  # 两者的requires_grad都是True
b.requires_grad_()


d = b * 3
d.backward()
print(b.grad) # b的梯度值为tensor(3.)
print(a.grad) # None


3232495738304 3232495737088
False False
tensor(3.)
None


In [79]:
# 采用detach()的效果 ## 被采用标记
a = torch.tensor(1.0, requires_grad=True)
b = a.clone().detach() # 割裂a和b为两个网络
print(a.data_ptr(), b.data_ptr())
print(a.requires_grad, b.requires_grad)
# (True False)  # 两者的requires_grad都是True
b.requires_grad_()

c = a * 2
c.backward()
print(a.grad)# a的梯度值为tensor(2.)

d = b * 3
d.backward()
print(b.grad) # b的梯度值为tensor(3.)

print(a.grad) # a的梯度值为tensor(2.)




3232495738304 3232495739392
True False
tensor(2.)
tensor(3.)
tensor(2.)
